In [ ]:
!pip install emoji==1.7.0
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=10badd1422f013f1346640d7a9b92f60a6f24dbd27b8105b38d486e63597cc26
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=3bc75c65df508e5947c6386c8206196f3b11adb6b52f7a97c094a8392be0bb00
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
!pip install tensorflow-text

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 83.0 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [ ]:
import re
import string

def clean_text(text):
    """
    This function cleans the input text by removing special characters,
    converting to lowercase, and removing extra spaces.
    """
    text = text.lower() # Convert to lowercase
    text = re.sub('\[.*?\]', '', text) # Remove text in square brackets
    text = re.sub('https?://\S+|www\.\S+', '', text) # Remove URLs
    text = re.sub('<.*?>+', '', text) # Remove HTML tags
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # Remove punctuation
    text = re.sub('\n', '', text) # Remove newline characters
    text = re.sub('\w*\d\w*', '', text) # Remove words containing numbers
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters except spaces
    text = re.sub(' +', ' ', text) # Remove extra spaces

    return text

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
!pip install tensorflow-text  # Install the tensorflow-text package

from tensorflow.keras.preprocessing.text import Tokenizer # Import Tokenizer from tensorflow.keras instead of keras
from tensorflow.keras.preprocessing.sequence import pad_sequences # Import pad_sequences from tensorflow.keras instead of keras
import numpy as np
import pandas as pd

# Load the data
train_data = pd.read_csv('fake_news_classification_mal_train.csv')

# Preprocess and clean text using the defined clean_text function
train_data['Cleaned_News'] = train_data['News'].apply(clean_text)

# Tokenize the text
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['Cleaned_News'])
X = tokenizer.texts_to_sequences(train_data['Cleaned_News'])
X = pad_sequences(X, padding='post', maxlen=100)

# Labels
y = pd.get_dummies(train_data['Label']).values

# Build the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=100, input_length=100))
model.add(LSTM(units=128, return_sequences=False))
model.add(Dense(5, activation='softmax'))  # 5 categories

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=5, batch_size=64)

# Predict on test data (after tokenizing and padding)
test_data = pd.read_csv('Fake_test_without_labels.csv')
test_data['Cleaned_News'] = test_data['text'].apply(clean_text)
X_test = tokenizer.texts_to_sequences(test_data['Cleaned_News'])
X_test = pad_sequences(X_test, padding='post', maxlen=100)

# Predict and save results
y_pred = model.predict(X_test)
predicted_labels = np.argmax(y_pred, axis=1)
test_data['Predicted_Label'] = predicted_labels

# Save predictions
test_data[['Id', 'Predicted_Label']].to_csv('test_predictions.csv', index=False)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


30/30 ━━━━━━━━━━━━━━━━━━━━ 9s 215ms/step - accuracy: 0.5619 - loss: 1.2868
Epoch 2/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 6s 186ms/step - accuracy: 0.6410 - loss: 1.1185
Epoch 3/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 10s 177ms/step - accuracy: 0.6241 - loss: 1.1386
Epoch 4/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 10s 177ms/step - accuracy: 0.6501 - loss: 1.0961
Epoch 5/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 11s 209ms/step - accuracy: 0.6419 - loss: 1.1109
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# True labels from the test set
y_test_true = pd.get_dummies(test_data['Predicted_Label']).values  # Assuming 'Label' contains the ground truth

# Predicted labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate accuracy
accuracy = accuracy_score(np.argmax(y_test_true, axis=1), y_pred_classes)
print(f"Accuracy: {accuracy}")

# Classification report for detailed metrics
print(classification_report(np.argmax(y_test_true, axis=1), y_pred_classes))
import pandas as pd

# Read the CSV file
df = pd.read_csv('test_predictions.csv')

Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1019

    accuracy                           1.00      1019
   macro avg       1.00      1.00      1.00      1019
weighted avg       1.00      1.00      1.00      1019



In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('test_predictions.csv')
# Save the DataFrame as a TSV file
df.to_csv('test_predictions.tsv', sep='\t', index=False)

In [ ]:
from google.colab import files
files.download('test_predictions.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def print_lines_with_1(file_path):
    """Prints lines from a file that contain the digit '1'."""
    with open(file_path, 'r') as file:
        for line in file:
            if '1' in line:  # Check if '1' is present in the line
                print(line, end='')  # Print the line (end='' prevents extra newline)

# Call the function with the file path
print_lines_with_1('test_predictions.tsv')

Fake_01	0
Fake_10	0
Fake_11	0
Fake_12	0
Fake_13	0
Fake_14	0
Fake_15	0
Fake_16	0
Fake_17	0
Fake_18	0
Fake_19	0
Fake_21	0
Fake_31	0
Fake_41	0
Fake_51	0
Fake_61	0
Fake_71	0
Fake_81	0
Fake_91	0
Fake_100	0
Fake_101	0
Fake_102	0
Fake_103	0
Fake_104	0
Fake_105	0
Fake_106	0
Fake_107	0
Fake_108	0
Fake_109	0
Fake_110	0
Fake_111	0
Fake_112	0
Fake_113	0
Fake_114	0
Fake_115	0
Fake_116	0
Fake_117	0
Fake_118	0
Fake_119	0
Fake_120	0
Fake_121	0
Fake_122	0
Fake_123	0
Fake_124	0
Fake_125	0
Fake_126	0
Fake_127	0
Fake_128	0
Fake_129	0
Fake_130	0
Fake_131	0
Fake_132	0
Fake_133	0
Fake_134	0
Fake_135	0
Fake_136	0
Fake_137	0
Fake_138	0
Fake_139	0
Fake_140	0
Fake_141	0
Fake_142	0
Fake_143	0
Fake_144	0
Fake_145	0
Fake_146	0
Fake_147	0
Fake_148	0
Fake_149	0
Fake_150	0
Fake_151	0
Fake_152	0
Fake_153	0
Fake_154	0
Fake_155	0
Fake_156	0
Fake_157	0
Fake_158	0
Fake_159	0
Fake_160	0
Fake_161	0
Fake_162	0
Fake_163	0
Fake_164	0
Fake_165	0
Fake_166	0
Fake_167	0
Fake_168	0
Fake_169	0
Fake_170	0
Fake_171	0
Fake_172	0
Fake_17